In [ ]:
!pip install transformers==2.9.1
!pip install kobert-transformers

In [ ]:
!git clone https://github.com/monologg/KoBERT-Transformers.git
!pip install KoBERT-Transformers/.

In [ ]:
from google.colab import files 
src = list(files.upload().values())[0] 
open('tokenization_kobert.py','wb').write(src)

Saving tokenization_kobert.py to tokenization_kobert.py


10994

In [ ]:
import json
import re
import numpy as np
import pandas as pd
import random
import time
import datetime

import tensorflow as tf
import torch

import tokenization_kobert
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
!git clone https://github.com/e9t/nsmc.git
!git clone https://github.com/SKTBrain/KoBERT.git

!pip install -r KoBERT/requirements.txt
!pip install KoBERT/.

In [ ]:
with open(path + "train_data.json", 'r') as json_file:
    train_data = json.load(json_file)
    
with open(path + "dev_data.json", 'r') as json_file:
    dev_data = json.load(json_file)

In [ ]:
train_x = []
train_Y = []
order = ['HS01', 'SS01', 'HS02', 'SS02', 'HS03', 'SS03']
RE_FILTER = re.compile("[.,!?\':;~()]")

for i in range(len(train_data)):
    for j in order:     
        text = train_data[i]['talk']['content'][j]
        text = re.sub(RE_FILTER,  ""  , text)
        train_x.append(text)
        
for i in range(len(train_data)):
    train_Y.append(train_data[i]['profile']['emotion']['type'])


train_X = []

for i in range(0, 94991, 6):
    X = train_x[i] + ' ' + train_x[i+1] + ' ' + train_x[i+2] + ' ' + train_x[i+3] + ' ' + train_x[i+4] + ' ' + train_x[i+5]
    train_X.append(X)

In [ ]:
dev_x = []
dev_Y = []
order = ['HS01', 'SS01', 'HS02', 'SS02', 'HS03', 'SS03']
RE_FILTER = re.compile("[.,!?\':;~()]")

for i in range(len(dev_data)):
    for j in order:     
        text = dev_data[i]['talk']['content'][j]
        text = re.sub(RE_FILTER,  ""  , text)
        dev_x.append(text)
        
for i in range(len(dev_data)):
    dev_Y.append(dev_data[i]['profile']['emotion']['type'])
    
dev_X = []

for i in range(0, 10301, 6):
    X = dev_x[i] + ' ' + dev_x[i+1] + ' ' + dev_x[i+2] + ' ' + dev_x[i+3] + ' ' + dev_x[i+4] + ' ' + dev_x[i+5]
    dev_X.append(X)

In [ ]:
# 대분류

train_Y_big = []
dev_Y_big = []

regex1 = re.compile('E1.'); regex2 = re.compile('E2.'); regex3 = re.compile('E3.') # 정규 표현식 
regex4 = re.compile('E4.'); regex5 = re.compile('E5.'); regex6 = re.compile('E6.')

for i in range(len(train_Y)):
    matchobj1 = regex1.match(train_Y[i]);matchobj2 = regex2.match(train_Y[i]);matchobj3 = regex3.match(train_Y[i]);
    matchobj4 = regex4.match(train_Y[i]);matchobj5 = regex5.match(train_Y[i]);matchobj6 = regex6.match(train_Y[i]);
    if matchobj1 != None:
        train_Y_big.append(0)
    elif matchobj2 != None:
        train_Y_big.append(1)
    elif matchobj3 != None:
        train_Y_big.append(2)
    elif matchobj4 != None:
        train_Y_big.append(3)   
    elif matchobj5 != None:
        train_Y_big.append(4)
    elif matchobj6 != None:
        train_Y_big.append(5)
    else:
        pass

for i in range(len(dev_Y)):
    matchobj1 = regex1.match(dev_Y[i]);matchobj2 = regex2.match(dev_Y[i]);matchobj3 = regex3.match(dev_Y[i]);
    matchobj4 = regex4.match(dev_Y[i]);matchobj5 = regex5.match(dev_Y[i]);matchobj6 = regex6.match(dev_Y[i]);
    if matchobj1 != None:
        dev_Y_big.append(0)
    elif matchobj2 != None:
        dev_Y_big.append(1)
    elif matchobj3 != None:
        dev_Y_big.append(2)
    elif matchobj4 != None:
        dev_Y_big.append(3)   
    elif matchobj5 != None:
        dev_Y_big.append(4)
    elif matchobj6 != None:
        dev_Y_big.append(5)
    else:
        pass

In [ ]:
sentences = train_X
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]

In [ ]:
from tokenization_kobert import KoBertTokenizer

tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

In [ ]:
MAX_LEN = 64

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [ ]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [ ]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    train_Y_big, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)


train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       train_Y_big,
                                                       random_state=2018, 
                                                       test_size=0.1)


train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [ ]:
batch_size = 64

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
sentences = dev_X
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]

tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(dev_Y_big)
test_masks = torch.tensor(attention_masks)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=60)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8 
                )

epochs = 10

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 10 ========
Training...


/usr/local/lib/python3.6/dist-packages/transformers/optimization.py:155: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)



  Average training loss: 2.15
  Training epcoh took: 0:01:32

Running Validation...
  Accuracy: 0.48
  Validation took: 0:00:03

======== Epoch 2 / 10 ========
Training...

  Average training loss: 1.31
  Training epcoh took: 0:01:32

Running Validation...
  Accuracy: 0.58
  Validation took: 0:00:03

======== Epoch 3 / 10 ========
Training...

  Average training loss: 1.11
  Training epcoh took: 0:01:32

Running Validation...
  Accuracy: 0.61
  Validation took: 0:00:03

======== Epoch 4 / 10 ========
Training...

  Average training loss: 1.01
  Training epcoh took: 0:01:32

Running Validation...
  Accuracy: 0.61
  Validation took: 0:00:03

======== Epoch 5 / 10 ========
Training...

  Average training loss: 0.91
  Training epcoh took: 0:01:32

Running Validation...
  Accuracy: 0.59
  Validation took: 0:00:03

======== Epoch 6 / 10 ========
Training...

  Average training loss: 0.82
  Training epcoh took: 0:01:32

Running Validation...
  Accuracy: 0.60
  Validation took: 0:00:03

=====

In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Accuracy: 0.58
Test took: 0:00:03
